<a href="https://colab.research.google.com/github/thrishul26/1_site/blob/main/vectorAddition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-emuzubry
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-emuzubry
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-wwv3ik6i/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [8]:
pip install nvcc4jupyter

In [9]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp70d_3fjz".


In [13]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

# // CUDA kernel. Each thread takes care of one element of c
__global__ void vecAdd(double *a, double *b, double *c, int n)
{
    # // Get our global thread ID
    int id = blockIdx.x*blockDim.x+threadIdx.x;

    # // Make sure we do not go out of bounds
    if (id < n)
        c[id] = a[id] + b[id];
}

int main( int argc, char* argv[] )
{
    # // Size of vectors
    int n = 1000000;

    # // Host input vectors
    double *h_a;
    double *h_b;
    # //Host output vector
    double *h_c;

    # // Device input vectors
    double *d_a;
    double *d_b;
    # //Device output vector
    double *d_c;

    # // Size, in bytes, of each vector
    size_t bytes = n*sizeof(double);

    # // Allocate memory for each vector on host
    h_a = (double*)malloc(bytes);
    h_b = (double*)malloc(bytes);
    h_c = (double*)malloc(bytes);

    # // Allocate memory for each vector on GPU
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    int i;
    # // Initialize vectors on host
    for( i = 0; i < n; i++ ) {
        h_a[i] = 1;
        h_b[i] = 1;
    }

    # // Copy host vectors to device
    cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);

    int blockSize, gridSize;

    # // Number of threads in each thread block
    blockSize = 1024;

    # // Number of thread blocks in grid
    gridSize = (int)ceil((float)n/blockSize);

    # // Execute the kernel
    vecAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);

    # // Copy array back to host
    cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );

    # // Sum up vector c and print result divided by n, this should equal 1 within error
    double sum = 0;
    for(i=0; i<n; i++)
        sum += h_c[i];
    printf("Vector additon: %f\n", sum);

    # // Release device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    # // Release host memory
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Vector additon: 2000000.000000

